# Difference in Differences

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from patsy import dmatrices
from statsmodels.stats.power import TTestIndPower

df = pd.read_stata('evaluation.dta')
df

,locality_identifier,household_identifier,treatment_locality,promotion_locality,eligible,enrolled,enrolled_rp,poverty_index,round,health_expenditures,...,educ_hh,educ_sp,female_hh,indigenous,hhsize,dirtfloor,bathroom,land,hospital_distance,hospital
0,26.0,5.0,1.0,1.0,1.0,1.0,1.0,55.950542,0.0,15.185455,...,0.0,6.0,0.0,0.0,4.0,1,0,1,124.819966,0.0
1,26.0,5.0,1.0,1.0,1.0,1.0,1.0,55.950542,1.0,19.580902,...,0.0,6.0,0.0,0.0,4.0,1,0,1,124.819966,0.0
2,26.0,11.0,1.0,1.0,1.0,1.0,0.0,46.058731,0.0,13.076257,...,4.0,0.0,0.0,0.0,6.0,1,0,2,124.819966,0.0
3,26.0,11.0,1.0,1.0,1.0,1.0,0.0,46.058731,1.0,2.398854,...,4.0,0.0,0.0,0.0,6.0,1,0,2,124.819966,1.0
4,26.0,13.0,1.0,1.0,1.0,1.0,0.0,54.095825,1.0,0.000000,...,0.0,0.0,0.0,0.0,6.0,1,0,4,124.819966,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19822,35.0,15738.0,0.0,0.0,0.0,0.0,0.0,59.737247,0.0,16.811539,...,0.0,2.0,0.0,1.0,7.0,0,1,2,162.748811,NaN
19823,40.0,15769.0,1.0,1.0,0.0,0.0,0.0,62.055641,0.0,15.906003,...,5.0,2.0,0.0,1.0,5.0,1,1,1,114.763392,NaN
19824,40.0,15769.0,1.0,1.0,0.0,0.0,0.0,62.055641,1.0,8.248152,...,5.0,2.0,0.0,1.0,5.0,1,1,1,114.763392,NaN
19825,40.0,15778.0,1.0,1.0,0.0,0.0,0.0,62.828438,0.0,8.737772,...,3.0,0.0,0.0,1.0,9.0,1,1,4,114.763392,NaN


In [8]:
# Display all variable names
print(df.columns)

Index(['locality_identifier', 'household_identifier', 'treatment_locality',
       'promotion_locality', 'eligible', 'enrolled', 'enrolled_rp',
       'poverty_index', 'round', 'health_expenditures', 'age_hh', 'age_sp',
       'educ_hh', 'educ_sp', 'female_hh', 'indigenous', 'hhsize', 'dirtfloor',
       'bathroom', 'land', 'hospital_distance', 'hospital'],
      dtype='object')


You are not very satisfied with the results of your analysis. Thinking more in-depth you realize that since you have data for two periods for each household in the sample, you can use these data to solve some of the challenges encountered in the previous cases. You will now compare the change in out-of-pocket health expenditures between the households that enrolled in the program and those that did not enroll. 

1.	Generate a new variable with the difference in out-of-pocket health expenditures between baseline (round = 0) and follow-up (round = 1). Call the new variable “delta_he”.

In [9]:
ID_COL = "household_identifier"

d = df.loc[df["treatment_locality"] == 1].copy()
d["round"] = d["round"].astype(int)

# ============================================================
# 1) delta_he = follow-up minus baseline per household
# ============================================================
wide = (
    d.pivot_table(index=ID_COL, columns="round", values="health_expenditures", aggfunc="mean")
      .rename(columns={0: "he_baseline", 1: "he_followup"})
)
wide["delta_he"] = wide["he_followup"] - wide["he_baseline"]

# attach delta back to long data (optional but handy)
d = d.merge(wide[["delta_he"]], left_on=ID_COL, right_index=True, how="left")

2.	Compare the average change in out-of-pocket health expenditures between the households that signed up for the program and those that didn’t in treatment localities. Assume we are in the random scenario, so make sure to use the variable enrolled_ro. 

In [10]:
# ============================================================
# 2) Compare mean change using the randomized promotion: enrolled_rp
#    (random scenario)
# ============================================================
if "enrolled_rp" in d.columns:
    rp = d[[ID_COL, "enrolled_rp"]].drop_duplicates(ID_COL).set_index(ID_COL)
    chg = wide.join(rp)
    g0 = chg.loc[chg["enrolled_rp"] == 0, "delta_he"].dropna()
    g1 = chg.loc[chg["enrolled_rp"] == 1, "delta_he"].dropna()

    print("== Mean delta_he by enrolled_rp ==")
    print(pd.DataFrame({
        "group": ["enrolled_rp=0", "enrolled_rp=1"],
        "mean_delta_he": [g0.mean(), g1.mean()],
        "n": [len(g0), len(g1)]
    }))

    tstat, pval, dfree = ttest_ind(g1, g0, usevar="unequal")
    print(f"Welch t-test (g1 - g0): t={tstat:.3f}, p={pval:.4f}, df≈{dfree:.1f}")
else:
    print("Column 'enrolled_rp' not found (check spelling).")


== Mean delta_he by enrolled_rp ==
           group  mean_delta_he     n
0  enrolled_rp=0       1.206617  2052
1  enrolled_rp=1      -6.593422  2907
Welch t-test (g1 - g0): t=-27.126, p=0.0000, df≈3231.5


3.	Run an OLS regression to estimate the effect of the program on out-of-pocket health expenditures using difference in differences: 
i.	Without controls
ii.	Including characteristics of the household head and spouse
iii.	Including baseline covariates


In [13]:
from patsy import dmatrices
import statsmodels.formula.api as smf
import numpy as np

def run_ols_cluster(formula, data, cluster_col):
    """
    Build the estimation sample with patsy (drops rows with NA in y/X),
    align the cluster groups to that exact sample, then fit with cluster-robust SEs.
    """
    # Build design matrices and drop rows with NA in y or X
    y, X = dmatrices(formula, data=data, return_type='dataframe', NA_action='drop')
    idx = y.index.intersection(X.index)

    # Analysis frame: exactly the rows kept by patsy + non-missing cluster ids
    analysis = data.loc[idx].dropna(subset=[cluster_col]).copy()

    # Fit and cluster
    model = smf.ols(formula, data=analysis)
    res = model.fit(cov_type="cluster", cov_kwds={"groups": analysis[cluster_col]})
    return res, analysis

In [14]:
# i) without controls
form_i = "health_expenditures ~ enrolled * round"
res_i, ana_i = run_ols_cluster(form_i, d, "locality_identifier")
print("\n== DID (i) — Without controls ==")
print(res_i.summary().tables[1])
print(f"DID effect (enrolled:round): {res_i.params.get('enrolled:round', np.nan):.3f}")


== DID (i) — Without controls ==
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         20.7915      0.172    120.678      0.000      20.454      21.129
enrolled          -6.3018      0.193    -32.638      0.000      -6.680      -5.923
round              1.5134      0.356      4.246      0.000       0.815       2.212
enrolled:round    -8.1629      0.319    -25.578      0.000      -8.788      -7.537
DID effect (enrolled:round): -8.163


In [15]:

# ii) + head & spouse characteristics (existing ones only)
hh_controls = [c for c in ["age_hh","age_sp","educ_hh","educ_sp","female_hh","hhsize"] if c in d.columns]
form_ii = "health_expenditures ~ enrolled * round" + ((" + " + " + ".join(hh_controls)) if hh_controls else "")
res_ii, ana_ii = run_ols_cluster(form_ii, d, "locality_identifier")
print("\n== DID (ii) — + Head/Spouse controls ==")
print("Controls used:", hh_controls if hh_controls else "(none)")
print(res_ii.summary().tables[1])
print(f"DID effect (enrolled:round): {res_ii.params.get('enrolled:round', np.nan):.3f}")



== DID (ii) — + Head/Spouse controls ==
Controls used: ['age_hh', 'age_sp', 'educ_hh', 'educ_sp', 'female_hh', 'hhsize']
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         24.6975      0.511     48.299      0.000      23.695      25.700
enrolled          -2.7804      0.139    -20.040      0.000      -3.052      -2.508
round              1.4435      0.356      4.055      0.000       0.746       2.141
enrolled:round    -8.1616      0.320    -25.530      0.000      -8.788      -7.535
age_hh             0.0852      0.011      7.641      0.000       0.063       0.107
age_sp            -0.0167      0.012     -1.399      0.162      -0.040       0.007
educ_hh            0.0570      0.036      1.576      0.115      -0.014       0.128
educ_sp            0.0415      0.035      1.185      0.236      -0.027       0.110
female_hh          1.0450      0.311      3.362 

In [16]:
# iii) + baseline covariates (values at round==0), plus head/spouse controls
base = d.loc[d["round"] == 0].copy()
exclude = {
    ID_COL, "round", "health_expenditures", "treatment_locality",
    "locality_identifier", "promotion_locality"
}
# numeric baseline covariates only
bnum = [c for c in base.columns if c not in exclude and np.issubdtype(base[c].dtype, np.number)]
b_pref = base[[ID_COL] + bnum].rename(columns={c: f"b_{c}" for c in bnum})
d3 = d.merge(b_pref, on=ID_COL, how="left")

b_cols = [c for c in d3.columns if c.startswith("b_")]
form_iii = "health_expenditures ~ enrolled * round"
if hh_controls: form_iii += " + " + " + ".join(hh_controls)
if b_cols:      form_iii += " + " + " + ".join(b_cols)

res_iii, ana_iii = run_ols_cluster(form_iii, d3, "locality_identifier")
print("\n== DID (iii) — + Baseline covariates (prefixed b_) + HH controls ==")
print("Baseline covariates used (first 12 shown):", b_cols[:12], "..." if len(b_cols) > 12 else "")
print(res_iii.summary().tables[1])
print(f"DID effect (enrolled:round): {res_iii.params.get('enrolled:round', np.nan):.3f}")



== DID (iii) — + Baseline covariates (prefixed b_) + HH controls ==
Baseline covariates used (first 12 shown): ['b_eligible', 'b_enrolled', 'b_enrolled_rp', 'b_poverty_index', 'b_age_hh', 'b_age_sp', 'b_educ_hh', 'b_educ_sp', 'b_female_hh', 'b_indigenous', 'b_hhsize', 'b_dirtfloor'] ...
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               6.8562      0.168     40.894      0.000       6.528       7.185
enrolled                6.8562      0.168     40.894      0.000       6.528       7.185
round                  -3.4397      0.155    -22.248      0.000      -3.743      -3.137
enrolled:round         -3.4397      0.155    -22.248      0.000      -3.743      -3.137
age_hh                  0.1238      0.234      0.530      0.596      -0.334       0.582
age_sp                  0.1353      0.064      2.116      0.034       0.010       0.261
educ_hh

/Users/justineloriaga/.julia/conda/3/aarch64/lib/python3.12/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 26, but rank is 20
  warnings.warn('covariance of constraints does not have full '


4.	What is the “effect” of the program on out-of-pocket health expenditures?

In [17]:
print("\n== Program effect (Difference-in-Differences, coef on enrolled:round) ==")
print(f"(i)   {res_i.params.get('enrolled:round', np.nan):.3f}")
print(f"(ii)  {res_ii.params.get('enrolled:round', np.nan):.3f}")
print(f"(iii) {res_iii.params.get('enrolled:round', np.nan):.3f}")


== Program effect (Difference-in-Differences, coef on enrolled:round) ==
(i)   -8.163
(ii)  -8.162
(iii) -3.440


DID improves upon Case 1 by:
- differencing out time-invariant household heterogeneity and common time shocks;
- focusing on within-household change over rounds and the interaction enrolled:round.

Remaining concerns:
- Parallel trends may fail if enrolled vs. non-enrolled would have evolved differently absent the program.
- Nonrandom take-up within treated localities; spillovers within locality clusters.
- Attrition or composition changes across rounds; measurement error in enrollment.
Consider checks: pre-trend balance, event-study, or IV using enrolled_rp for take-up.
